In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import gymnasium as gym
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
#from data_utils import get_nth_episode_info
from IPython.display import clear_output
from sapien.core import Pose
from mani_skill2.envs.pick_and_place.base_env import StationaryManipulationEnv
from mani_skill2.utils.registration import register_env
import gymnasium as gym
import sapien.core as sapien
from mani_skill2 import ASSET_DIR
from pathlib import Path
from typing import Dict, List
from mani_skill2 import format_path
from mani_skill2.utils.io_utils import load_json
import numpy as np
from sapien.core import Pose
import matplotlib.pyplot as plt
from mani_skill2.agents.base_agent import BaseAgent
import os
import warnings
# Register ManiSkill2 environments in gym
import mani_skill2.envs
from mani_skill2 import ASSET_DIR
from mani_skill2.utils.sapien_utils import look_at
from mani_skill2.utils.registration import register_env
from mani_skill2.envs.pick_and_place.pick_cube import PickCubeEnv
from mani_skill2.utils.sapien_utils import hex2rgba
from mani_skill2.sensors.camera import CameraConfig
from transforms3d.euler import euler2quat, quat2euler
from mani_skill2.utils.sapien_utils import vectorize_pose
from mani_skill2.agents.base_controller import BaseController
from IPython.display import Video
import shutil
import random
from plot_utils import *
from config import color_maps, background_quadrants

In [4]:
from utils import *
from agent_env_utils import *
color_maps = {
    "red": (1, 0, 0),
    "blue": (0, 0, 1),
    "green": (0, 1, 0),
    "white": (1, 1, 1),
    "black": (0, 0, 0),
    "yellow": (1, 1, 0),
    "orange": (1, 0.5, 0),
    "purple": (0.5, 0, 0.5),
    "gray": (0.5, 0.5, 0.5),
}
directions = ["front", "behind", "right", "left", "top"]

In [6]:
import copy
def create_directories(data_dir, empty=False):
    final_dir = f"{data_dir}/final"
    os.makedirs(final_dir, exist_ok=True)
    if empty:
        shutil.rmtree(final_dir)
        os.makedirs(final_dir, exist_ok=True)
    return final_dir

def finalize_quadrant(quadrant, scale=1):
    lim_x = quadrant[0]
    lim_y = quadrant[1]
    interval_x = [0, lim_x]
    interval_y = [0, lim_y]
    interval_x.sort()
    interval_y.sort()
    interval_x = np.array(interval_x)*scale
    interval_y = np.array(interval_y)*scale
    return interval_x, interval_y

# Define directories
data_dir = "../benchmark/multiview"
final_dir = create_directories(data_dir, empty=True)

# Initialize pair of colors
color_keys = color_maps.keys()
color_pairs = list(itertools.permutations(color_keys, 3))
random.seed(42)
random.shuffle(color_pairs)

# Define ranges and object parameters
object_shapes = ["sphere", "cube"]
object_size_ranges = {
    "sphere": (0.05, 0.02),
    "cube": (0.05, 0.08),
    "random_sphere": (0.01, 0.05)
}
object_position_range = [(-0.3, 0.3), (-0.3, 0.25)]
num_random_balls = 5

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")
    directions = list(env.direction_placement_map.keys())
    
    # Print available directions
    print(f"Available directions: {env.direction_placement_map.keys()}")
    outputs = []
    for background in trange(4):
        for quadrant_idx, quadrant in enumerate(background_quadrants[background]):
            # Decide the boundary of each background
            interval_x, interval_y = finalize_quadrant(quadrant, scale=0.6)
            object_position_range = [interval_x, interval_y]
            
            for color_idx in trange(5):  # range(len(color_pairs)):
                color_1, color_2, color_3 = color_pairs[color_idx]
                for r in range(5):
                    size = np.random.uniform(0.05, 0.08, 1)[0]
                    cube1_config = {'size': size, 'color': color_maps[color_1], 'obj_type': "cube", 'ranges': [(-0.3, 0.3), (-0.3, 0.0)], 'static': True}
                    cube2_config = {'size': size, 'color': color_maps[color_2], 'obj_type': "cube", 'ranges': [(-0.3, 0.3), (-0.3, 0.25)], 'static': True}
                    sphere1_config = {'size': 0.05, 'color': color_maps[color_3], 'obj_type': "sphere", 'ranges': [(-0.3, 0.3), (-0.3, 0.25)], 'static': True}

                    configs = [cube1_config, cube2_config, sphere1_config]
                    configs_collate = collate_infos(configs)
                    env.register_configures(configs_collate)
                    _ = env.reset(options={"reconfigure": True})

                    env.initialize_objects(background=background)
                    obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))

                    initial_poses = env.get_important_obj_poses(mode="all")  # Store initial poses

                    # -----------------
                    for i, direction in enumerate(directions):
                        if direction not in env.direction_placement_map:
                            print(f"Skipping invalid direction: {direction}")
                            continue
                        
                        env.set_poses(initial_poses)  # Restore initial positions before placement
                        
                        if (background == 2 and direction == "right"):
                            reverseDirection = "left"
                            env.place_cubes_in_direction([(0, 1, reverseDirection)])
                            env.place_cubes_in_direction([(2, 0, "top")])
                        elif (background == 2 and direction == "left"):
                            reverseDirection = "right"
                            env.place_cubes_in_direction([(0, 1, reverseDirection)])
                            env.place_cubes_in_direction([(2, 0, "top")])
                        elif (background == 2 and direction == "front"):
                            reverseDirection = "behind"
                            env.place_cubes_in_direction([(0, 1, reverseDirection)])
                            env.place_cubes_in_direction([(2, 0, "top")])
                        elif (background == 2 and direction == "behind"):
                            reverseDirection = "front"
                            env.place_cubes_in_direction([(0, 1, reverseDirection)])
                            env.place_cubes_in_direction([(2, 0, "top")])
                        else:
                            env.place_cubes_in_direction([(0, 1, direction)])
                            env.place_cubes_in_direction([(2, 0, "top")])
                        
                        obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
                        poses_initial = env.get_important_obj_poses()

                        # Save the correct view (front, side, top)
                        save_dir_final = f"{final_dir}/{r}_{color_1}_{direction}_{color_2}_bg{background}_q{quadrant_idx}.png"
                        collect_and_save(env, save_dir_final, steps=1, mode=[f"front{background}", f"side{background}", f"top{background}"])

                        # Set the answer_path to point to the correct top view image
                        answer_path = f"{final_dir}/{r}_{color_1}_{direction}_{color_2}_bg{background}_q{quadrant_idx}_top.png"

                        poses_final = env.get_important_obj_poses(mode="all")
                        # ----------------
                        # Wrong Choices
                        wrong_choices = []
                        
                        # (3) Placing Sphere on Blue Cube (spatial relationship)
                        wrong_placement_config = configs_collate.copy()  # Randomly select one cube to put on top of the sphere
                        env.register_configures(wrong_placement_config)
                        swapped_poses = poses_final.copy()
                        swapped_poses = [swapped_poses[2], swapped_poses[1], swapped_poses[0]]
                        env.set_poses(swapped_poses)  # Restore initial positions
                        #env.place_cubes_in_direction([(0, 2, "top")])  # Place the chosen cube on top of the sphere
                        wrong_placement_save_dir = f"{final_dir}/{r}_{color_1}_{direction}_{color_2}_bg{background}_q{quadrant_idx}_top_choice3.png"
                        collect_and_save(env, wrong_placement_save_dir, steps=1, mode=[f"top{background}"])
                        wrong_choices.append(wrong_placement_save_dir)


                        # (1) Swapping Colors (attribute)
                        swapped_color_config = configs_collate.copy()
                        swapped_color_config['color'][0], swapped_color_config['color'][1] = swapped_color_config['color'][1], swapped_color_config['color'][0]
                        env.register_configures(swapped_color_config)
                        _ = env.reset(options={"reconfigure": True}) # only swap color need
                        env.set_poses(poses_final)
                        # env.initialize_objects(background=background)
                        swapped_color_save_dir = f"{final_dir}/{r}_{color_1}_{direction}_{color_2}_bg{background}_q{quadrant_idx}_top_choice1.png"
                        collect_and_save(env, swapped_color_save_dir, steps=1, mode=[f"top{background}"])
                        wrong_choices.append(swapped_color_save_dir)
                        
                        # (2) Changing Size of Sphere (attribute)
                        changed_size_config = configs_collate.copy()
                        changed_size_config['size'][2] = 0.1  # Change size of the sphere
                        #Change back the colors from last choice config
                        changed_size_config['color'][0], changed_size_config['color'][1] = changed_size_config['color'][1], changed_size_config['color'][0]
                        env.register_configures(changed_size_config)
                        _ = env.reset(options={"reconfigure": True}) 
                        env.set_poses(poses_final)  # Keep original cube positions
                        changed_size_save_dir = f"{final_dir}/{r}_{color_1}_{direction}_{color_2}_bg{background}_q{quadrant_idx}_top_choice2.png"
                        collect_and_save(env, changed_size_save_dir, steps=1, mode=[f"top{background}"])
                        wrong_choices.append(changed_size_save_dir)


                        # Restore the correct configuration and placements
                        env.set_poses(initial_poses)  # Restore initial poses after wrong choices

                        # Save the output with the answer_path
                        outputs.append({
                            "colors": configs_collate['color'],
                            "answer_path": answer_path,  # The top view answer path
                            "wrong_choices": wrong_choices,  # Wrong choices paths
                            "obj_types": configs_collate['obj_type'],
                            "round": r,
                            "sizes": configs_collate['size'],
                            "poses_final": [list(np.float64(item)) for item in poses_initial],
                        })

    write_out(f"{data_dir}/index.jsonl", outputs)

del env


[2024-09-17 08:42:16.546] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.


Available directions: dict_keys(['behind', 'front', 'left', 'right', 'top'])


  0%|          | 0/4 [00:00<?, ?it/s]